# Mini project: Simulating Pac-Man

In the last practical session, we saw how run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating behavior activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simple implementations of the [Braitenberg vehicles](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame1550) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other epucks:

In [ ]:
close_session(simulator)

In [3]:
from numpy.linalg import norm 
from numpy import array
from simulator_interface import open_session, close_session 

simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)
class ScenarioWrapper : 
    def __init__(self, robot1, robot2, robot3): 
        self.robot1 = robot1 
        self.robot2 = robot2
        self.robot3 = robot3

    def getNearestRobot(self,robot):
        a = norm(array(robot.position())-array(self.robot1.position()));
        b = norm(array(robot.position())-array(self.robot2.position()));
        c = norm(array(robot.position())-array(self.robot3.position()));
        if self.isMiddle(a,b,c):
            return self.robot2
        if self.isMiddle(b,a,c):
            return self.robot1
        return self.robot3
    
    def isMiddle(self,a,b,c):
        return (b>=a and c>=b) or (b>=c and a>=b)
    
rumble=ScenarioWrapper(epuck1, epuck2, epuck3)

#Reverse-engineered part of the code in order to make spheres disappear when eaten
def eat(robot): 
    robot_pos = array(robot.position()) 
    for obj in simulator.eatable_objects: 
        try: 
            obj_pos = array(simulator.get_object_position(obj))

        #if robot is self we are not interested
            if norm(robot_pos - obj_pos) < 0.05 :
                continue
        except:
            break
        if norm(robot_pos - obj_pos) < 0.1 :
            simulator.eatable_objects.remove(obj)
            #eat cuboid -ª remove from scene and increases score of the robot
            if obj.startswith('Cuboid'):
                robot.num_eats = robot.num_eats+1
                simulator.remove_object(obj)
            else:
                #if the collision is with a robot we compare robot score to decide which of them wins the battle
                rival=rumble.getNearestRobot(robot)
                if rival.num_eats < robot.num_eats:
                    rival.dead=True
            robot.rotate = 'no'

# First reduce the maximum speed
for e in simulator.robots:
    e.max_speed = 3
    e.num_eats = 0
    e.dead = False

def park(robot):
    print ('Parking ' + str(robot.robot_id))
    robot.left_spd=3
    robot.right_spd=0
    robot.wait(4)
    robot.left_spd=3
    robot.right_spd=3
    robot.wait(8)
    robot.left_spd=0
    robot.right_spd=0
def retire(robot):    
    print ('robot unavailable ' + str(robot.robot_id))
    robot.left_spd = 0
    robot.right_spd = 0
    robot.stop_all_behaviors()
    robot.detach_all_behaviors()
    robot.stop_all_routines()
    print ('score: ' + str(robot.num_eats))
    park(robot)
    
def line_following(robot):
    floor_left, floor_middle, floor_right = robot.floor_sensor()
    # if can go right
    if floor_middle == 1.0 and floor_right == 1.0 and robot.rotate == 'right':
        return  [1.0, 0]
    # if can go left
    if floor_middle == 1.0 and floor_left == 1.0 and robot.rotate == 'left':
        return  [0, 1.0]
    if floor_middle == 1.0: return [.3, .3]
    if floor_left == 0.0 and floor_right == 0.0: return [0.1, 0]
    return [floor_right/2.0, floor_left/2.0]

def rotate_by_sensor(robot, left, right):
    if left>right: 
            if left-right>0.1:
                robot.rotate='left'
            else:
                return
    if right>left:
        if right-left>0.1:
            robot.rotate='right'
        else:
            return

def rotate_by_sensor_eats(robot, left, right, left_num_eats, right_num_eats):
    if left>right: 
            if left-right>0.1:
                robot.rotate='left' if left_num_eats < robot.num_eats else 'right'
            else:
                robot.rotate=robot.rotate if left_num_eats < robot.num_eats else 'right'
    if right>left:
        if right-left>0.1:
            robot.rotate = 'right' if right_num_eats < robot.num_eats else 'left'
        else:
            robot.rotate=robot.rotate if right_num_eats < robot.num_eats else 'left'

def detect_food(robot):
    left, right = robot.prox_activations(tracked_objects=['Cuboid'])
    if (left > 0 or right > 0):
        rotate_by_sensor(robot, left, right)
                
def detect_dead(robot):
    if robot.dead:
        retire(robot)
        
def detect_robot(robot):
    (left, right), (epuck_left, epuck_right) = robot.prox_activations(tracked_objects=["ePuck"], return_epucks=True)
    left_num_eats, right_num_eats = robot.sensed_epuck_attributes(epuck_left, epuck_right, "num_eats", default_value = 0)
    left_dead, right_dead = robot.sensed_epuck_attributes(epuck_left, epuck_right, "dead", default_value = False)
   
    if left>0 :
        if right>0 :
            if left_dead :
                if right_dead:
                    return
                else:
                    robot.rotate = 'right' if right_num_eats < robot.num_eats else 'left' 
            else:
                if right_dead:
                    robot.rotate = 'left' if left_num_eats < robot.num_eats else 'right'
                else:
                    rotate_by_sensor_eats(robot, left, right,left_num_eats, right_num_eats)
        else:
            if left_dead :
                return
            else:
                robot.rotate = 'left' if left_num_eats < robot.num_eats else 'right'
    else:
        if right>0 :
            if right_dead:
                    return
            else:
                robot.rotate = 'right' if right_num_eats < robot.num_eats else 'left' 
        else:
            return
            
   
        

simulator.eatable_objects = ['Cuboid4','Cuboid0', 'Cuboid1', 'Cuboid2', 'Cuboid3', 'ePuck', 'ePuck#1', 'ePuck#0']
for e in simulator.robots:
    e.detach_all_behaviors()
    e.attach_routine(eat, freq=1)
    e.start_routine(eat)
########

for e in simulator.robots:
    e.detach_all_behaviors()
    e.rotate = 'no'
    e.attach_behavior(line_following, freq=10)
    e.attach_routine(detect_food, freq=1)
    e.attach_routine(detect_robot, freq=1)
    e.attach_routine(detect_dead, freq=1)
    e.start_routine(detect_food)
    e.start_routine(detect_robot)
    e.start_routine(detect_dead)
    e.start_all_behaviors()

#nearest=rumble.getNearestRobot(epuck1) 
#print(epuck1.robot_id)
#print(nearest.robot_id)


Routine eat started
Routine eat started
Routine eat started
Routine detect_food started
Routine detect_robot started
Routine detect_dead started
Behavior line_following started
Routine detect_food started
Routine detect_robot started
Routine detect_dead started
Behavior line_following started
Routine detect_food started
Routine detect_robot started
Routine detect_dead started
Behavior line_following started
robot unavailable 2
Behavior line_following stopped
Routine eat stopped
Routine detect_food stopped
Routine detect_robot stopped
Routine detect_dead stopped
score: 0
Parking 2
robot unavailable 1
Behavior line_following stopped
Routine eat stopped
Routine detect_food stopped
Routine detect_robot stopped
Routine detect_dead stopped
score: 1
Parking 1


In [4]:
close_session(simulator)

In [ ]:
close_all_connections()